In [ ]:
# Model 4: Dynamic Expansion
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image

class BaseStateSystem:
    pass

def laplacian1D(data, dx):
    """Computes a 1D Laplacian using finite differences."""
    lap = np.zeros_like(data)
    lap[1:-1] = (data[2:] - 2 * data[1:-1] + data[:-2]) / (dx**2)
    return lap

def central_apical_initialiser(shape):
    """Start with one apical cell in the center, left grows first, right is delayed."""
    a = np.zeros(shape)
    b = np.zeros(shape)

    center = shape // 2
    left = center - 10
    right = center + 10
    
    a[center] = 0.5
    a[left] = 0.2
    a[right] = 0.05

    inhibition = np.ones(shape)
    inhibition[right:] = 0.05
    inhibition[:left] = 0.9

    return a, b, inhibition, center, right

class OneDimensionalRDEquations(BaseStateSystem):
    def __init__(
        self, Da, Db, Ra, Rb,
        initialiser=central_apical_initialiser,
        width=500, dx=1,
        dt=0.1, steps=1,
        expansion_amount=5, # the amount that you expand per expansion
        edge_threshold = 0.1,
        expansion_threshold = 50
    ):
        self.Da = Da
        self.Db = Db
        self.Ra = Ra
        self.Rb = Rb
        self.initialiser = initialiser
        self.width = width
        self.dx = dx
        self.dt = dt
        self.steps = steps
        self.expand_axis = False
        self.current_width = width
        self.max_width = 2000  # Maximum width for the x-axis
        self.expansion_counter = 0  # Counter to track iterations since last expansion
        self.edge_threshold = edge_threshold # if the value exceeds this, then increase
        self.expansion_threshold = expansion_threshold  # Iterations before another expansion
        self.expansion_amount = expansion_amount

    def initialise(self):
        self.t = 0
        self.a, self.b, self.inhibition, self.center, self.right_index = self.initialiser(self.width)

    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()
            if self.should_expand_axis():
                self.expansion_counter += 1
                if self.expansion_counter >= self.expansion_threshold:
                    self.expand_arrays()
                    self.expansion_counter = 0

    def _update(self):
        a, b = self.a, self.b
        Da, Db = self.Da, self.Db
        Ra, Rb = self.Ra, self.Rb
        dt, dx = self.dt, self.dx

        La = laplacian1D(a, dx)
        Lb = laplacian1D(b, dx)

        delta_a = dt * (Da * La + Ra(a, b) * self.inhibition)
        delta_b = dt * (Db * Lb + Rb(a, b))

        self.a += delta_a
        self.b += delta_b

        left_threshold = 0.4
        if np.max(a[: self.center]) > left_threshold:
            self.inhibition[self.right_index:] += 0.001

        self.inhibition = np.clip(self.inhibition, 0, 1)


    def should_expand_axis(self):
        """
        Checks if the axis should be expanded based on the activator (A) reaching the edges.
        """
        edge_region_size = 20  # Check the outermost 10 grid points on each side
        left_edge = self.a[:edge_region_size]
        right_edge = self.a[-edge_region_size:]

        # If any value in the edge regions exceeds the threshold, return True
        if np.any(left_edge > self.edge_threshold) or np.any(right_edge > self.edge_threshold):
            return True
        return False

    def expand_arrays(self):
        expansion = self.expansion_amount #  A larger expansion step
        new_width = min(self.current_width + expansion, self.max_width)  # Respect max_width
        if new_width > self.current_width:
            new_a = np.zeros(new_width)
            new_b = np.zeros(new_width)
            new_inhibition = np.ones(new_width)

            start = (new_width - self.current_width) // 2
            end = start + self.current_width

            new_a[start:end] = self.a
            new_b[start:end] = self.b
            new_inhibition[start:end] = self.inhibition

            # Add noise in new regions
            noise_level = 0.001  # Reduced noise level
            new_a[:start] = noise_level * np.random.rand(start)
            new_b[:start] = noise_level * np.random.rand(start)
            new_inhibition[:start] = 1

            new_a[end:] = noise_level * np.random.rand(new_width - end)
            new_b[end:] = noise_level * np.random.rand(new_width - end)
            new_inhibition[end:] = 1

            self.a = new_a
            self.b = new_b
            self.inhibition = new_inhibition
            self.current_width = new_width

            #update center/right
            self.center = new_width // 2
            self.right_index = self.center + 10 # or whatever formula you use


    def draw(self, ax):
        ax.clear()
        x = np.linspace(-self.current_width//2, self.current_width//2, self.current_width)
        ax.plot(x, self.a, color="r", label="A (Activator)")
        ax.plot(x, self.b, color="b", label="B (Inhibitor)")
        ax.legend()
        ax.set_ylim(-1, 1)
        ax.set_xlim(-self.current_width//2, self.current_width//2)
        ax.set_title(f"t = {self.t:.2f}")

    def plot_time_evolution(self, filename, n_steps=800, fps=20):
        self.initialise()

        fig, ax = plt.subplots(figsize=(12, 6))

        def update_frame(i):
            self.update()
            self.draw(ax)

        anim = FuncAnimation(
            fig,
            update_frame,
            frames=n_steps,
            interval=50,
            repeat=False
        )

        writer = PillowWriter(fps=fps)
        anim.save(filename, writer=writer)
        plt.close(fig)

# Usage
Da, Db, alpha, beta = 1, 100, -0.005, 10

def Ra(a, b): return a - a**3 - b + alpha
def Rb(a, b): return (a - b) * beta

width = 500
dx = 1
dt = 0.001

system = OneDimensionalRDEquations(
    Da, Db, Ra, Rb,
    initialiser=central_apical_initialiser,
    width=width,
    dx=dx,
    dt=dt,
    steps=250,
    expansion_amount = 5,
    edge_threshold = 0.1,
    expansion_threshold = 50
)

system.plot_time_evolution("1dRD_central_apical_continuous_growth.gif", n_steps=800)

Image("1dRD_central_apical_continuous_growth.gif")